# 1 задание

In [2]:
from google.colab import files

uploaded = files.upload()

Saving corpus_hum.txt to corpus_hum.txt


In [0]:
import csv
csv_path = "data_paraphraser_norm.csv"
texts_1, texts_2 = [], []
classes = []
with open(csv_path, "r") as f:
  reader = csv.DictReader(f)
  for line in reader:
    texts_1.append(line["text_1_norm"])
    texts_2.append(line["text_2_norm"])
    classes.append(line["label"])

In [4]:
len(classes)

7227

In [0]:
import re
def clear_sent(sent):
  return ' '.join([re.sub('_.*', '', w) for w in sent.split()])
texts_1_no_tags = [clear_sent(s) for s in texts_1]
texts_2_no_tags = [clear_sent(s) for s in texts_2]

In [6]:
print(texts_1[0])
print(texts_1_no_tags[0])

полицейский_NOUN разрешать_VERB стрелять_VERB на_ADP поражение_NOUN по_ADP гражданин_NOUN с_ADP травматика_NOUN
полицейский разрешать стрелять на поражение по гражданин с травматика


In [8]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.2MB/s 


In [11]:
import nltk
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
from string import punctuation
from nltk.corpus import stopwords
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
data = open('corpus_hum.txt').read().splitlines()

data_norm = [normalize(text) for text in data]
data_norm = [text for text in data_norm if text]
data_norm[:5]

['абай василий васо иван 1900–2001 русский лингвист родиться 2 15 декабрь 1900 с.коби тифлисский губерния ныне грузия 1925 окончить факультет общественный наука ленинградский университет 1928 аспирантура 1928–1930 сотрудник кавказский историко-археологический институт ан ссср 1930 полвека работать яфетический институт затем институт язык мышление институт языкознание ан ссср ленинград 1950 москва доктор филологический наука 1962 профессор 1969 лауреат государственный премия ссср 1981 почётный член азиатский королевский общество великобритания ирландия 1966 член-корреспондент финно-угорский общество хельсинки 1973 умереть абай москва 18 март 2001',
 'также тема',
 'лингвистика языкознание языковедение',
 'ученик н.я.марра ранний работа находиться влияние идея который впоследствии отойти видный специалист история иранский язык свой родной осетинский язык автор фундаментальный историко-этимологический словарь осетинский язык 5-ти том 1958–1990 заниматься также мифология фольклор иранский 

In [13]:
import gensim
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)
w2v.most_similar('язык')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('древнееврейский', 0.8031230568885803),
 ('иврит', 0.8012332320213318),
 ('латынь', 0.7776556015014648),
 ('церковно-славянский', 0.7645127177238464),
 ('украинск', 0.7639732360839844),
 ('фарси', 0.7615458965301514),
 ('идиш', 0.7530549168586731),
 ('фольклор', 0.7513177990913391),
 ('по-арабски', 0.7499111294746399),
 ('осетинский', 0.743545413017273)]

In [14]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
!gunzip news_upos_cbow_300_2_2017.bin.gz

--2020-02-19 08:11:01--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz’

news_upos_cbow_300_ 100%[===================>] 133.81M  28.5MB/s    in 5.4s    

2020-02-19 08:11:07 (24.6 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz’ saved [140310878/140310878]



In [15]:
from gensim.test.utils import datapath
w2v_external = gensim.models.KeyedVectors.load_word2vec_format(datapath("/content/news_upos_cbow_300_2_2017.bin"), binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from collections import Counter,defaultdict
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [24]:
import numpy as np
dim = 50
X_text_1_w2v = np.zeros((len(texts_1_no_tags), dim))
X_text_2_w2v = np.zeros((len(texts_2_no_tags), dim))

for i in range(len(texts_1_no_tags)):
    X_text_1_w2v[i] = get_embedding(texts_1_no_tags[i], w2v, dim)
    
for i in range(len(texts_2_no_tags)):
    X_text_2_w2v[i] = get_embedding(texts_2_no_tags[i], w2v, dim)

X_text_1_w2v_external = np.zeros((len(texts_1), dim))
X_text_2_w2v_external = np.zeros((len(texts_2), dim))

for i in range(len(texts_1)):
    X_text_1_w2v_external[i] = get_embedding(texts_1[i], w2v_external, dim)
    
for i in range(len(texts_2)):
    X_text_2_w2v_external[i] = get_embedding(texts_2[i], w2v_external, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)
X_text_w2v_external = np.concatenate([X_text_1_w2v_external, X_text_2_w2v_external], axis=1)
y = classes

In [26]:
from sklearn import linear_model
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
n_fold = 10
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
f1 = 0
for train_index, valid_index in stratified_folds.split(X_text_w2v, y):
    train_texts = [X_text_w2v[i] for i in train_index]
    train_results = [y[i] for i in train_index]
    test_texts = [X_text_w2v[i] for i in valid_index]
    test_results = [y[i] for i in valid_index]
    model = linear_model.LogisticRegression(class_weight='balanced')
    model.fit(train_texts, train_results)
    preds = model.predict(test_texts)
    f1_fold= f1_score(test_results, preds, average='weighted')
    f1 += f1_fold
f1_external = 0
stratified_folds_external = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
for train_index, valid_index in stratified_folds_external.split(X_text_w2v_external, y):
    train_texts = [X_text_w2v_external[i] for i in train_index]
    train_results = [y[i] for i in train_index]
    test_texts = [X_text_w2v_external[i] for i in valid_index]
    test_results = [y[i] for i in valid_index]
    model = linear_model.LogisticRegression(class_weight='balanced')
    model.fit(train_texts, train_results)
    preds = model.predict(test_texts)
    f1_fold = f1_score(test_results, preds, average='micro')
    f1_external += f1_fold
  
print("my w2v: ", f1/n_fold)
print("rusvectores w2v: ", f1_external/n_fold)

my w2v:  0.4186536288337878
rusvectores w2v:  0.3572717554970633


Обученная модель показала себя лучше, чем взятая с rusvectores

# 2 задание

In [0]:
ft = gensim.models.FastText([text.split() for text in data_norm], size=50, 
                                   min_n=4, max_n=8) 

In [22]:
ft.most_similar('язык')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('н.м.язык', 0.9947590231895447),
 ('праязык', 0.9452675580978394),
 ('подъязык', 0.916778564453125),
 ('язык-предок', 0.8909441232681274),
 ('языкова', 0.8596363067626953),
 ('метаязык', 0.8408960700035095),
 ('язык-цель', 0.8406948447227478),
 ('протоязык', 0.8103739619255066),
 ('языковед', 0.7974911332130432),
 ('диалектолог', 0.755706787109375)]

In [23]:
X_text_1_ft = np.zeros((len(texts_1_no_tags), dim))
X_text_2_ft = np.zeros((len(texts_2_no_tags), dim))
for i in range(len(texts_1_no_tags)):
    X_text_1_ft[i] = get_embedding(texts_1_no_tags[i], ft, dim)
    
for i in range(len(texts_2_no_tags)):
    X_text_2_ft[i] = get_embedding(texts_2_no_tags[i], ft, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(texts_1 + texts_2)

In [0]:
from sklearn.decomposition import TruncatedSVD, NMF
svd = TruncatedSVD(200)
svd = svd.fit(X)
X_text_1_svd = svd.transform(cv.transform(texts_1))
X_text_2_svd = svd.transform(cv.transform(texts_2))

In [0]:
nmf = NMF(50)
nmf = nmf.fit(X)
X_text_1_nmf = nmf.transform(cv.transform(texts_1))
X_text_2_nmf = nmf.transform(cv.transform(texts_2))

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
def micro_f1_for_multiple_models(models):
  distances = []
  for i in range(len(y)):
    d = []
    for model in models:
      d.append(cosine_similarity(model[0][i].reshape(1, -1), model[1][i].reshape(1, -1)))
    distances.append(d)
  reshaped_distances = []
  for d in distances:
    r_d = []
    for item in d:
      r_d.append(item[0][0])
    reshaped_distances.append(r_d)
  n_fold = 10
  f1_multi = 0
  stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
  for train_index, valid_index in stratified_folds.split(reshaped_distances, y):
    train_texts = [reshaped_distances[i] for i in train_index]
    train_results = [y[i] for i in train_index]
    test_texts = [reshaped_distances[i] for i in valid_index]
    test_results = [y[i] for i in valid_index]
    model = linear_model.LogisticRegression(class_weight='balanced')
    model.fit(train_texts, train_results)
    preds = model.predict(test_texts)
    f1_fold= f1_score(test_results, preds, average='micro')
    f1_multi += f1_fold
  return f1_multi/n_fold

In [31]:
models = [[X_text_1_svd,X_text_2_svd],
          [X_text_1_nmf,X_text_2_nmf],
          [X_text_1_ft,X_text_2_ft],
          [X_text_1_w2v,X_text_2_w2v],
          [X_text_1_w2v_external,X_text_2_w2v_external]]
print(micro_f1_for_multiple_models(models))

0.4813833557468688


Результат стал лучше, попробуем поменять параметры у методов векторизации

In [32]:
min_dfs = [1,3,5]
max_dfs = [0.2,0.4,0.6]
max_features = [500,1000,1500]
for min_df in min_dfs:
  for max_df in max_dfs:
    for max_feature in max_features:
      new_cv = CountVectorizer(min_df=min_df, max_df=max_df, max_features=max_feature)
      new_X = new_cv.fit_transform(texts_1 + texts_2)
      new_svd = TruncatedSVD(200)
      new_svd = new_svd.fit(new_X)
      new_nmf = NMF(50)
      new_nmf = new_nmf.fit(new_X)
      new_X_text_1_svd = new_svd.transform(new_cv.transform(texts_1))
      new_X_text_2_svd = new_svd.transform(new_cv.transform(texts_2))
      new_X_text_1_nmf = new_nmf.transform(new_cv.transform(texts_1))
      new_X_text_2_nmf = new_nmf.transform(new_cv.transform(texts_2))
      new_models = [
          [new_X_text_1_svd,new_X_text_2_svd],
          [new_X_text_1_nmf,new_X_text_2_nmf],
          [X_text_1_ft,X_text_2_ft],
          [X_text_1_w2v,X_text_2_w2v],
          [X_text_1_w2v_external,X_text_2_w2v_external]]
      print("min_df: ", min_df, " max_df: ", max_df, " max_features: ", max_feature, " micro_f1: ", micro_f1_for_multiple_models(new_models))

min_df:  1  max_df:  0.2  max_features:  500  micro_f1:  0.4765408826718467
min_df:  1  max_df:  0.2  max_features:  1000  micro_f1:  0.47681731627605817
min_df:  1  max_df:  0.2  max_features:  1500  micro_f1:  0.4751562242579587
min_df:  1  max_df:  0.4  max_features:  500  micro_f1:  0.4802768550553059
min_df:  1  max_df:  0.4  max_features:  1000  micro_f1:  0.47986134258993196
min_df:  1  max_df:  0.4  max_features:  1500  micro_f1:  0.47654030796580893
min_df:  1  max_df:  0.6  max_features:  500  micro_f1:  0.4844275736294219
min_df:  1  max_df:  0.6  max_features:  1000  micro_f1:  0.4823534595387793
min_df:  1  max_df:  0.6  max_features:  1500  micro_f1:  0.4834601517990215
min_df:  3  max_df:  0.2  max_features:  500  micro_f1:  0.47571139029053305
min_df:  3  max_df:  0.2  max_features:  1000  micro_f1:  0.4758491281709406
min_df:  3  max_df:  0.2  max_features:  1500  micro_f1:  0.47224993582449254
min_df:  3  max_df:  0.4  max_features:  500  micro_f1:  0.4809665023007398

Наилучшие результаты при CountVectorizer(min_df=1, max_df=0.6, max_features=500) - 0.4844275736294219

In [33]:
best_cv = CountVectorizer(min_df=1, max_df=0.6, max_features=500)
best_X = best_cv.fit_transform(texts_1 + texts_2)
best_svd = TruncatedSVD(200)
best_svd = best_svd.fit(best_X)
best_nmf = NMF(50)
best_nmf = best_nmf.fit(best_X)
best_X_text_1_svd = best_svd.transform(best_cv.transform(texts_1))
best_X_text_2_svd = best_svd.transform(best_cv.transform(texts_2))
best_X_text_1_nmf = best_nmf.transform(best_cv.transform(texts_1))
best_X_text_2_nmf = best_nmf.transform(best_cv.transform(texts_2))

sgs = [0, 1]
hss = [0, 1]
sizes = [25,50,75,100]
for sg in sgs:
  for hs in hss:
    for size in sizes:
      new_w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=size, sg=sg, hs=hs)
      new_X_text_1_w2v = np.zeros((len(texts_1_no_tags), dim))
      new_X_text_2_w2v = np.zeros((len(texts_2_no_tags), dim))

      for i in range(len(texts_1_no_tags)):
        new_X_text_1_w2v[i] = get_embedding(texts_1_no_tags[i], new_w2v, dim)
    
      for i in range(len(texts_2_no_tags)):
        new_X_text_2_w2v[i] = get_embedding(texts_2_no_tags[i], new_w2v, dim)


      new_models = [
          [best_X_text_1_svd,best_X_text_2_svd],
          [best_X_text_1_nmf,best_X_text_2_nmf],
          [X_text_1_ft,X_text_2_ft],
          [new_X_text_1_w2v,new_X_text_2_w2v],
          [X_text_1_w2v_external,X_text_2_w2v_external]]
      print("sg: ", sg, " hs: ", hs, " size: ", size, " micro_f1: ", micro_f1_for_multiple_models(new_models))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


sg:  0  hs:  0  size:  25  micro_f1:  0.4699039857779413


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


sg:  0  hs:  0  size:  50  micro_f1:  0.4692106987276009


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


...

sg:  1  hs:  1  size:  50  micro_f1:  0.49065777021720053


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


sg:  1  hs:  1  size:  75  micro_f1:  0.4699039857779413


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


sg:  1  hs:  1  size:  100  micro_f1:  0.4699039857779413


Наилучшие результаты при Word2Vec(size=50, sg=1, hs=1) - 0.49065777021720053 (на самом деле возможен и результат лучше т.к. в прошлый раз при size=50, sg=1, hs=0 был результат 0.4844275736294219, а сейчас он 0.4824927299686211)
Параметры fasttext перебирать не стал т.к. долго делается